## AFLUX: AFLOW Search-API

**Aim:** Programmatically expose the same functionality as our web search interface at https://aflow.org/search

- AFLUX enables search through the query string of the URL ```?<operator>``` by exposing several logical operators.
- Operators can be inter-property and/or intra-property.

### List of logical operators and AFLUX syntax
|Logical operator |      AFLUX syntax          |        Operator             |
| --------------  |:--------------------------:| ---------------------------:|
| ```<block>```   |    ```(``` and ```)```     |  Intra- and inter-property  |
| ```<AND>```     |          ```,```           |  Intra- and inter-property  |
| ```<OR>```      |          ```:```           |  Intra- and inter-property  |
| ```<NOT>```     |          ```!```           |       Intra-property        |
| ```<loose>```   |          ```*```           |       Intra-property        |
| ```<string>```  |          ```'```           |       Inter-property        |
| ```<mute>```    |          ```$```           |       Intra-property        |

- In AFLUX operators are used to create a query. In our terminology a query is comprised of a **matchbook** and **directives**.

    ![](../images/AFLUX_URL.png)

### Matchbook:
- Materials keywords with arguments: ```<server>?species((Na:K),Cl),nspecies(2),Egap(1*,*5),energy_cell```.

### Directives:
- Used for formatting, moving between layers (catalog) and pagination.
- **format** takes args ```"json"``` and ```"html"```.
- **catalog** takes args ```"icsd"```, ```"lib1"```, ```"lib2"```, ...
- **paging** controls page number, number of entries per page, and sorting. By default entries are sorted in ascending order of first materials keyword. Order can be reversed by using a negative page number.


## AFLUX: examples

Help directives

In [ ]:
# Help text.
!curl 'https://aflow.org/API/aflux/?'

In [ ]:
# List of matchbook keywords.
!curl 'https://aflow.org/API/aflux/?schema,format(json)'

Operators \<block\>, \<AND\>, \<OR\>, \<NOT\>

In [ ]:
# Search compounds containing Cr and 3 total species.
!curl 'https://aflow.org/API/aflux/?species(Cr),catalog(ICSD),nspecies(3),paging(1)'

In [ ]:
# Exclude oxygen from the results.
!curl 'https://aflow.org/API/aflux/?species(Cr,!O),catalog(ICSD),nspecies(3),paging(1)'

In [ ]:
# List compounds from ICSD catalog containing Cr or Mn and 3 species.
!curl 'https://aflow.org/API/aflux/?species(Cr:Mn),catalog(ICSD),nspecies(3),paging(1)'

In [ ]:
# Retrieve compounds with 3 species from ICSD catalog containing Cr and Se, or Cr and Te and 3 species.
!curl 'https://aflow.org/API/aflux/?species(Cr,(Se:Te)),catalog(ICSD),nspecies(3),paging(1)'

Operators: \<loose\>

In [ ]:
# Retrieve the band gaps for all quaternary compounds
!curl 'https://aflow.org/API/aflux/?Egap,nspecies(4),paging(1)'

In [ ]:
# Only show non-null results
!curl 'https://aflow.org/API/aflux/?Egap(*),nspecies(4),paging(1)'

In [ ]:
# Only show results where Egap is greater than or equal to 1eV
!curl 'https://aflow.org/API/aflux/?Egap(1*),nspecies(4),paging(1)'

In [ ]:
# Only show results where Egap is greater than or equal to 1eV and less than or equal to 3eV
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),paging(1)'

Directives: paging

In [ ]:
# Show nth (e.g. 5th) results page (default: 64 entries/page):
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),paging(5)'

In [ ]:
# Change page size to m (e.g. 100) entries per page:
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),paging(5,100)'

Sorting

In [ ]:
# Sort in ascending order:
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),$paging(1)'

In [ ]:
# Sort in descending order (also works for n=0):
!curl 'https://aflow.org/API/aflux/?Egap(1*,*3),nspecies(4),$paging(-1)'

Aliases

In [ ]:
# Search for metal chalcogenides:
!curl 'https://aflow.org/API/aflux/?spacegroup_relax(216),Pearson_symbol_relax(cF8),nspecies(2),species(Chalcogens,Metals),paging(0)'

The following code block enables the quick interaction with AFLUX using python.

In [1]:
import json
from json import JSONDecodeError
from urllib.request import urlopen

SERVER="https://aflow.org"
API="/API/aflux/?"

# Download a AFLUX response and return it as list of dictionaries
def aflux_request(matchbook, paging=1, no_directives=False):
    request_url = SERVER + API + matchbook
    if not no_directives:
        request_url += f",$paging({paging}),format(json)"
    server_response = urlopen(request_url)
    response_content  = server_response.read().decode("utf-8")
    # Basic error handling
    if server_response.getcode() == 200:
        try:
            return json.loads(response_content)
        except JSONDecodeError:
            pass
    print("AFLUX request failed!")
    print(f"  URL: {request_url}")
    print(f"  Response: {response_content}")
    return []

# Print the build in help of AFLUX
def aflux_help(keyword=None):
    if keyword is None:
        # General help (https://aflow.org/API/aflux/?)
        help_data = aflux_request("", no_directives=True)
        print("\n".join(help_data))
    else:
        # Help regarding a specific keyword (https://aflow.org/API/aflux/?help(keyword))
        help_data = aflux_request(f"help({keyword})")
        for key, entry in help_data.items():
            print(key)
            print(f"  description: {entry['description']}")
            print(f"  units: {entry['units']}")
            print(f"  status: {entry['status']}")
            comment = "\n    ".join(entry["__comment__"]).strip()
            if comment:
                print(f"  comment:\n    {comment}")

### 1. Example
From the ICSD catalog, find the VRH bulk moduli ('ael_bulk_modulus_vrh') for materials containing Ti in the AFLOW database. What is the material with the highest bulk modulus? What is the lattice type ('Bravais_lattice_relax') and space group of this material? Is it a metal or an insulator ('Egap_type')? 

In [ ]:
aflux_help('ael_bulk_modulus_vrh')

In [ ]:
data_vrh = aflux_request('species(Ti),catalog(icsd),ael_bulk_modulus_vrh,Bravais_lattice_relax')

In [ ]:
data_vrh

In [4]:
data_vrh = aflux_request('species(Ti),catalog(icsd),ael_bulk_modulus_vrh,Bravais_lattice_relax')

In [5]:
data_vrh

[{'compound': 'Ag4Cs4S8Ti2',
  'auid': 'aflow:a8cd34ff24baeccf',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/Ag2Cs2S4Ti1_ICSD_280645',
  'spacegroup_relax': 132,
  'Pearson_symbol_relax': 'tP18',
  'species': ['Ag', 'Cs', 'S', 'Ti'],
  'catalog': 'ICSD',
  'ael_bulk_modulus_vrh': None,
  'Bravais_lattice_relax': 'TET'},
 {'compound': 'Ag4Eu4O16Ti4',
  'auid': 'aflow:3e25083b7929dd89',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/TET/Ag1Eu1O4Ti1_ICSD_78720',
  'spacegroup_relax': 57,
  'Pearson_symbol_relax': 'oP28',
  'species': ['Ag', 'Eu', 'O', 'Ti'],
  'catalog': 'ICSD',
  'ael_bulk_modulus_vrh': None,
  'Bravais_lattice_relax': 'ORC'},
 {'compound': 'Ag4Eu4O16Ti4',
  'auid': 'aflow:8228e72e9fcbf3c7',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/ICSD_WEB/ORC/Ag1Eu1O4Ti1_ICSD_78720',
  'spacegroup_relax': 57,
  'Pearson_symbol_relax': 'oP28',
  'species': ['Ag', 'Eu', 'O', 'Ti'],
  'catalog': 'ICSD',
  'ael_bulk_modulus_vrh': None,
  'Bravais_lattice_relax': 'ORC'},
 {'compound'

2. Determine the bulk moduli of all binary materials in the AFLOW database with a transition metal and a chalcogen and a band gap between 1 eV and 3 eV. For the five compounds with the lowest bulk moduli, list the chemical formulae, space groups, bulk moduli, and band gaps.

In [ ]:
"""
    Sample Python script to query AFLUX server
"""
#!/usr/bin/env python
import json, sys, os
from urllib.request import urlopen

SERVER="https://aflow.org"
API="/API/aflux/?"
MATCHBOOK=""
DIRECTIVES=""
SUMMONS=MATCHBOOK+","+DIRECTIVES

response=json.loads(urlopen(SERVER+API+SUMMONS).read().decode("utf-8"))
for datum in response:
    

3. Narrow the choice of transition metals down to Fe, Ni and Co. Determine the five compounds with the largest magnetic moments per atom ('spin_atom') and list their chemical formulae, space groups, bulk moduli, and magnetic moments per atom.

In [ ]:
"""
    Sample Python script to query AFLUX server
"""
#!/usr/bin/env python
import json, sys, os
from urllib.request import urlopen

SERVER="http://aflow.org"
API="/API/aflux/?"
MATCHBOOK=""
DIRECTIVES=""
SUMMONS=MATCHBOOK+","+DIRECTIVES

response=json.loads(urlopen(SERVER+API+SUMMONS).read().decode("utf-8"))
for datum in response:
    